In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/01/30 20:10:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/01/30 20:10:55 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
green_folderpath = 'data/pq/green/*/*'

df_green = spark.read.parquet(green_folderpath)

In [5]:
cols = [
    'VendorID',
    'lpep_pickup_datetime',
    'PULocationID',
    'DOLocationID',
    'trip_distance'
]

duration_rdd = df_green \
    .select(cols) \
    .rdd

In [7]:
import pandas as pd

def predict(df):
    pred = df.trip_distance * 5
    return pred

def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=cols)
    preds = predict(df)
    df['predicted_duration'] = preds
    
    for row in df.itertuples():
        yield row

In [10]:
df_predicted = duration_rdd \
    .mapPartitions(apply_model_in_batch) \
    .toDF() \
    .drop('Index')

In [12]:
df_predicted \
    .select('VendorID', 'trip_distance', 'predicted_duration') \
    .show()

+--------+-------------+------------------+
|VendorID|trip_distance|predicted_duration|
+--------+-------------+------------------+
|       2|         0.94| 4.699999999999999|
|       2|         1.42|               7.1|
|       2|         4.87|             24.35|
|       2|         1.43|7.1499999999999995|
|       2|         0.64|               3.2|
|       2|         5.64|              28.2|
|       2|         2.86|14.299999999999999|
|       2|         0.48|               2.4|
|       2|          4.0|              20.0|
|       2|         2.16|              10.8|
|       2|         3.71|             18.55|
|       2|         1.06| 5.300000000000001|
|       2|         0.82|               4.1|
|       2|        13.45|             67.25|
|       2|         3.53|             17.65|
|       2|         1.56| 7.800000000000001|
|       2|         7.07|             35.35|
|       1|          2.0|              10.0|
|       2|         1.38|6.8999999999999995|
|       2|         3.97|        